In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [7]:
train_dir = '../data/processed/asl_alphabet_train_cropped/asl_alphabet_train'
test_dir = '../data/processed/asl_alphabet_test_cropped/asl_alphabet_test'

img_height = 100
img_width = 100
batch_size = 32

In [ ]:
import os
import shutil

test_dir = '../data/raw/asl_alphabet_test/asl_alphabet_test'

for filename in os.listdir(test_dir):
    if filename.endswith('.jpg'):
        class_name = filename.split('_')[0]

        class_folder = os.path.join(test_dir, class_name)
        if not os.path.exists(class_folder):
            os.makedirs(class_folder)

        old_path = os.path.join(test_dir, filename)
        new_path = os.path.join(class_folder, filename)
        shutil.move(old_path, new_path)

print("✅ Test images have been rearranged into class folders!")


In [8]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,        
    width_shift_range=0.2,    
    height_shift_range=0.2,   
    shear_range=0.2,          
    zoom_range=0.2,           
    horizontal_flip=True,     
    fill_mode='nearest',      
    brightness_range=[0.8, 1.2],  # New addition: Random brightness adjustment
    channel_shift_range=20.0  # New addition: Color channel shifting
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 69600 images belonging to 29 classes.
Found 17400 images belonging to 29 classes.
Found 28 images belonging to 28 classes.


In [ ]:
from tensorflow.keras import layers, models


model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(12800, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_generator.num_classes, activation='softmax')
])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 98, 98, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 49, 49, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 47, 47, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 23, 23, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 21, 21, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 10, 10, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 12800)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 12800)          │   163,852,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 12800)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 29)             │       371,229 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 164,317,277 (626.82 MB)

 Trainable params: 164,317,277 (626.82 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau

optimizer = Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])


lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[lr_scheduler, early_stopping]
)



Epoch 1/10


c:\Users\CHPde\OneDrive\Bureau\Projects\Sign Language Project\ADS-Project\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2175/2175 ━━━━━━━━━━━━━━━━━━━━ 280s 128ms/step - accuracy: 0.1367 - loss: 2.9641 - val_accuracy: 0.3560 - val_loss: 2.1572
Epoch 2/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 182s 64ms/step - accuracy: 0.4561 - loss: 1.7103 - val_accuracy: 0.5018 - val_loss: 1.5930
Epoch 3/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 119s 55ms/step - accuracy: 0.5729 - loss: 1.3017 - val_accuracy: 0.5629 - val_loss: 1.4327
Epoch 4/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 118s 54ms/step - accuracy: 0.6411 - loss: 1.0884 - val_accuracy: 0.5894 - val_loss: 1.3010
Epoch 5/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 165s 76ms/step - accuracy: 0.6812 - loss: 0.9527 - val_accuracy: 0.6217 - val_loss: 1.2091
Epoch 6/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 148s 68ms/step - accuracy: 0.7164 - loss: 0.8516 - val_accuracy: 0.6513 - val_loss: 1.0899
Epoch 7/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 182s 84ms/step - accuracy: 0.7420 - loss: 0.7763 - val_accuracy: 0.6242 - val_loss: 1.2101
Epoch 8/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 390s 179ms/step - accuracy: 0.7590 - 

In [6]:
model.save('../models/sign_language_model_cropped32.h5')

In [ ]:
import matplotlib.pyplot as plt

# Extract metrics from the history object
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

# Plot accuracy
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()